In [ ]:
import numpy as np
# reference: https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb
# reference: https://github.com/Nir-J/Decision_tree_ID3/blob/master/dtree.ipynb

In [ ]:
def is_numeric(val):
  return isinstance(val,int) or isinstance(val,float)
header = ["color", "diameter", "label"]

#Question

In [ ]:
class Question:
  """ compare the given value """
  def __init__(self,column,value):
    self.column = column
    self.value = value

  def match(self,example):
    val = example[self.column]
    if is_numeric(val):
      return val >= self.value
    else:
      return val == self.value

  def __repr__(self):
    condition = "=="
    if is_numeric(self.value):
      condition = ">="
    return "Is %s %s %s?"%(header[self.column],condition,str(self.value))

In [ ]:
print(Question(1,3))
print(Question(1,"Apple"))

Is diameter >= 3?
Is diameter == Apple?


# Partition

In [ ]:
def partition(rows,question):
  """ partition a dataset """
  true_rows,false_rows = [],[]
  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
  return true_rows,false_rows

In [ ]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
# partition according to the question given
true_rows,false_rows = partition(training_data,Question(0,"Red"))
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


# Gini

In [ ]:
def class_counts(rows):
  counts = {}
  for row in rows:
    label = row[-1]
    if label not in counts:
      counts[label] = 0
    counts[label] += 1
  return counts


In [ ]:
def gini(rows):
  """ calculate gini impurity: how impure is our label class"""
  counts = class_counts(rows)
  impurity = 1
  for lbl in counts:
    prob_of_lbl = counts[lbl] / float(len(rows))
    impurity -= prob_of_lbl**2
  return impurity




In [ ]:
test = [["Apple"],
     ["Apple"]]
print("very pure labels:%f"%gini(test))
test2= [["Apple"],
    ["Orange"]]
print("half pure labels:%f"%gini(test2))
print("disorder labels of different kinds:%.4f"%gini(training_data))

very pure labels:0.000000
half pure labels:0.500000
disorder labels of different kinds:0.6400


In [ ]:
def info_gain_gini(left,right,current_uncertainty):
  p = float(len(left)) / (len(left)+len(right))
  return current_uncertainty - p*gini(left) -(1-p)*gini(right)

# how much information do we gain by partioning on "Green" ?
true_rows,false_rows = partition(training_data,Question(0,"Green"))
# initial uncertainty at root node
current_uncertainty = gini(training_data) 
info_gain_gini(true_rows,false_rows,current_uncertainty)

0.1399999999999999

# Entropy

In [ ]:
def entropy(rows):
  counts = class_counts(rows)
  entropy = 0
  for lbl in counts:
    prob_of_lbl = -counts[lbl]/float(len(rows))*np.log2(counts[lbl]/float(len(rows)))
    entropy -= prob_of_lbl
  return entropy
def info_gain_entropy(left,right,current_uncertainty):
  p = float(len(left))/(len(left)+len(right)) # percent of left
  return current_uncertainty - p*entropy(left) -(1-p)*entropy(right)
current_uncertainty = entropy(training_data)
info_gain_entropy(true_rows,false_rows,current_uncertainty)

-0.32192809488736196

# find best split

In [ ]:
def find_best_split(rows):
  best_gain = 0
  best_question = None
  current_uncertainty = gini(rows)
  n_features = len(rows[0])-1 # number of columns

  for col in range(n_features):
    values = set([row[col] for row in rows])

    for val in values:
      question = Question(col,val)
      
      true_rows,false_rows = partition(rows,question)

      if len(true_rows) ==0 or len(false_rows) == 0:
        continue

      gain = info_gain_gini(true_rows,false_rows,current_uncertainty)

      if gain>= best_gain:
        best_gain,best_question = gain,question
  return best_gain,best_question

# Leaf and Tree

In [ ]:
class Leaf:
  def __init__(self,rows):
    self.predictions = class_counts(rows)

In [ ]:
class Node:
  def __init__(self,question,true_branch,false_branch):
    self.question = question
    self.true_branch = true_branch
    self.false_branch = false_branch

In [ ]:
def build_tree(rows):
  gain,question = find_best_split(rows)
  
  if gain == 0:
    return Leaf(rows)

  true_rows,false_rows = partition(rows,question)

  true_branch = build_tree(true_rows)

  false_branch = build_tree(false_rows)

  return Node(question,true_branch,false_branch)

In [ ]:
def print_tree(node,spacing=""):

  if isinstance(node,Leaf):
    print(spacing+"Predict",node.predictions)
    return
  
  print(spacing + str(node.question))

  print(spacing + "--> True:")
  print_tree(node.true_branch,spacing+" ")
  
  print(spacing + "--> False:")
  print_tree(node.false_branch,spacing+" ")

In [ ]:
my_tree = build_tree(training_data)

In [ ]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
 Is color == Yellow?
 --> True:
  Predict {'Apple': 1, 'Lemon': 1}
 --> False:
  Predict {'Apple': 1}
--> False:
 Predict {'Grape': 2}


In [ ]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)